# Introduction
See https://github.com/wtsi-team112/fits/issues/4

Magnus sent an email 17/09/2018 14:57 with 13 "unexplained" files that are in fits with sample ID of a sample in Pf6.2 manifest, but the file itself is not in Pf6.2 manifest. I suspect that these files might have been sequenced in "invalid" studies. I have considered a file to be an invalid study if, after mapping from sequencescape to alfresco studies using ```/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/sequencescape_alfresco_study_mappings.txt```, the Alfresco study code does not start with four integers. An example of a sequencescape study that fall in this category is study 1935 ("AT-rich amplification"). In this notebook, I determine which files have been excluded due to having an "invalid" study.

In [1]:
%run ../setup.ipynb

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.13.3
scipy 0.19.1
pandas 0.22.0
numexpr 2.6.4
pysam 0.8.4
pysamstats 0.24.3
petlx 1.0.3
vcf 0.6.8
h5py 2.7.0
tables 3.4.2
zarr 2.2.0
scikit-allel 1.1.10


In [2]:
# Inputs
mlwh_missing_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_missing_exceptions.txt'
mlwh_study_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_study_exceptions.txt'
mlwh_sample_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_sample_exceptions.txt'
mlwh_taxon_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_taxon_exceptions.txt'
sequencescape_alfresco_study_mappings_fn='/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/sequencescape_alfresco_study_mappings.txt'

# Outputs
# output_fn = 'Pf_62_Pv4_like_manifest_20180917.txt'
# manifest_symlink = 'pf_62_manifest.txt'
input_dir = '/lustre/scratch118/malaria/team112/pipelines/setups/pf_62/input'

# Create assay data table for MalariaGEN samples

In [3]:
def set_irods_name(row):
    # First calculate prefix using nonhuman dependent on id_run and species (taxon)
    # Then calculate tag string (empty string if no tag)
    # Then calculate file extension
    # Then concat these three strings
    if (
        (row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 6750)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    elif (
        ~(row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 7100)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    else:
        prefix = "%d_%d" % (row['id_run'], row['position'])
        
    if np.isnan(row['tag_index']):
        tag_string = ''
    else:
        tag_string = '#%d' % row['tag_index']
    
    if row['id_run'] <= 10300:
        file_extension = '.bam'
    else:
        file_extension = '.cram'
    
    irods_filename = prefix + tag_string + file_extension
    return(irods_filename)

def set_sample_id(row):
    if row['sample'] is None:
        return('DS_%s' % row['name'])
    else:
        return('DS_%s' % row['sample'])

def is_valid_alfresco_code(s):
    try: 
        int(s[0:4])
        return True
    except ValueError:
        return False

def create_build_manifest(
#     taxon_ids=[5833, 5855, 5858, 5821, 7165, 7173, 30066], # Pf, Pv, Pm, Pb, Ag, A. arabiensis, A. merus
    taxon_ids=[5833, 36329, 5847, 57267, 137071], # Pf, 3D7, V1, Dd2, HB3
    # 1089 is excluded as this is R&D samples. 1204 and 1176 are excluded as these are CP1 samples. 1175 is excluded as some Pf R&D samples were incorrectly tagged with this study
    studies_to_exclude = ['1089-R&amp;D-GB-ALCOCK', '1204-PF-GM-CP1', '1176-PF-KE-CP1', '1175-VO-KH-STLAURENT'],
    miseq_runs_to_allow = [13809, 13810], # Two Miseq runs within 24 samples on each from study 1095-PF-TZ-ISHENGOMA that were included in Pf 6.0
    input_dir = input_dir,
    mlwh_missing_exceptions_fn = mlwh_missing_exceptions_fn,
    mlwh_study_exceptions_fn = mlwh_study_exceptions_fn,
    mlwh_sample_exceptions_fn = mlwh_sample_exceptions_fn,
    mlwh_taxon_exceptions_fn = mlwh_taxon_exceptions_fn,
    sequencescape_alfresco_study_mappings_fn = sequencescape_alfresco_study_mappings_fn,
    output_columns = [
        'path',
        'study',
        'sample',
        'lane',
        'reads',
        'paired',
        'irods_path',
        'sanger_sample_id',
        'taxon_id',
        'study_lims',
        'study_name',
        'id_run',
        'position',
        'tag_index',
        'qc_complete',
        'manual_qc',
        'description',
        'instrument_name',
        'instrument_model',
        'forward_read_length',
        'requested_insert_size_from',
        'requested_insert_size_to',
        'human_percent_mapped',
        'subtrack_filename',
        'subtrack_files_bytes',
        'ebi_run_acc'
    ]
):
    """Create a DataFrame to be used as a build manifest.
    
    A build manifest here is a list of all the "lanelets" that need to be
    included in a build. The output will typically be written to a
    tab-delmited file, either to use as input to vr-pipe, or perhaps as
    input to a vr-track DB.

    Args:
        taxon_ids (list of int): The taxons of the build (P. falciparum is '5833',
            P. vivax is '5855').
        sequencescape_alfresco_study_mappings_fn (str): filename of
            mappings from sequencscape to alfresco study mappings

    Returns:
        pd.DataFrame: The build manifest.

    """
    
    # Read in taxon exceptions file
    df_mlwh_taxon_exceptions = pd.read_csv(mlwh_taxon_exceptions_fn, sep='\t', dtype={'tag_index': str, 'taxon_id': int}, index_col='irods_filename')
    df_mlwh_taxon_exceptions['tag_index'].fillna('', inplace=True)
    # Identify which samples in exceptions file match the taxon of current build, and which don't
    df_mlwh_exceptions_this_taxon = df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)]
    df_mlwh_exceptions_other_taxa = df_mlwh_taxon_exceptions.loc[~(df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids))]

    # Read in sample exceptions file
    df_mlwh_sample_exceptions = pd.read_csv(mlwh_sample_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in study exceptions file
    df_mlwh_study_exceptions = pd.read_csv(mlwh_study_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in missing exceptions file
    df_mlwh_missing_exceptions = pd.read_csv(mlwh_missing_exceptions_fn, sep='\t', index_col='irods_filename')
    df_mlwh_missing_exceptions['derivative_sample_id'] = 'DS_' + df_mlwh_missing_exceptions['sample_id']
    df_mlwh_missing_exceptions = df_mlwh_missing_exceptions.loc[
        df_mlwh_missing_exceptions['taxon_id'].isin(taxon_ids)
    ]

    # Read in SequenceScape-Alfresco study mappings
    df_sequencescape_alfresco_study_mappings = pd.read_csv(sequencescape_alfresco_study_mappings_fn, sep='\t', index_col='seqscape_study_id')
    sequencescape_alfresco_study_mappings_dict = df_sequencescape_alfresco_study_mappings.loc[
        :,
#         df_sequencescape_alfresco_study_mappings['build_flag']==1,
        'alfresco_study_code'
    ].to_dict()
    
    # Read in data from mlwh matching this taxon, plus samples from exceptions file matching this taxon
    conn = pymysql.connect(
        host='mlwh-db',
        user='mlwh_malaria',
        password='Solaris&2015',
        db='mlwarehouse',
        port=3435
    )
    
    sql_query = 'SELECT \
        study.name as study_name, \
        study.id_study_lims as study_lims, \
        sample.supplier_name as sample, \
        sample.name, \
        sample.sanger_sample_id, \
        sample.taxon_id, \
        iseq_product_metrics.id_run, \
        iseq_product_metrics.position, \
        iseq_product_metrics.tag_index, \
        iseq_product_metrics.num_reads, \
        iseq_product_metrics.human_percent_mapped, \
        iseq_run_lane_metrics.instrument_name, \
        iseq_run_lane_metrics.instrument_model, \
        iseq_run_lane_metrics.paired_read, \
        iseq_run_lane_metrics.qc_complete, \
        iseq_flowcell.manual_qc, \
        iseq_flowcell.requested_insert_size_from, \
        iseq_flowcell.requested_insert_size_to, \
        iseq_flowcell.forward_read_length, \
        iseq_run_status_dict.description \
    FROM \
        study, \
        iseq_flowcell, \
        sample, \
        iseq_product_metrics, \
        iseq_run_status, \
        iseq_run_lane_metrics, \
        iseq_run_status_dict \
    WHERE \
        study.id_study_tmp = iseq_flowcell.id_study_tmp and \
        iseq_flowcell.id_sample_tmp = sample.id_sample_tmp and \
        iseq_flowcell.manual_qc = 1 and \
        iseq_product_metrics.id_iseq_flowcell_tmp = iseq_flowcell.id_iseq_flowcell_tmp and \
        iseq_run_status.id_run = iseq_product_metrics.id_run and \
        iseq_product_metrics.id_run = iseq_run_lane_metrics.id_run and \
        iseq_product_metrics.position = iseq_run_lane_metrics.position and \
        iseq_run_status.iscurrent = 1 and \
        ( ( iseq_run_lane_metrics.instrument_model != "MiSeq" ) or (iseq_product_metrics.id_run in (%s)) ) and \
        iseq_run_status.id_run_status_dict = iseq_run_status_dict.id_run_status_dict and \
        study.faculty_sponsor = "Dominic Kwiatkowski" and \
        ( ( sample.taxon_id in (%s) ) or ' % (
            ', '.join([str(x) for x in miseq_runs_to_allow]),
            ', '.join([str(x) for x in taxon_ids])
        )
    sql_query = sql_query + ' or '.join(
        df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)].apply(
            lambda x: '(iseq_product_metrics.id_run="%s" and iseq_product_metrics.position="%s" and iseq_product_metrics.tag_index="%s")' % (x['id_run'], x['position'], x['tag_index']),
            1
        )
    )
    sql_query = sql_query + ')'
#         (iseq_flowcell.manual_qc = 1 or iseq_flowcell.manual_qc is null) and \
    
    df_return = pd.read_sql(sql_query, conn)

    # Replace missing taxon_id with -1 (can't have missing int values in pandas Series)
    df_return['taxon_id'] = df_return['taxon_id'].fillna(-1).astype('int32')

    # Determine file name in iRods
    df_return['irods_filename'] = df_return.apply(set_irods_name, 1)
    df_return.set_index('irods_filename', inplace=True)

    # Determine alfresco study and change any incorrect studies
    df_return['alfresco_study_code'] = df_return['study_lims'].apply(
        lambda x: sequencescape_alfresco_study_mappings_dict[int(x)] if int(x) in sequencescape_alfresco_study_mappings_dict else ''
    )
    empty_alfresco_study_code = (df_return['alfresco_study_code'] == '')
    df_return.loc[empty_alfresco_study_code, 'alfresco_study_code'] = df_return.loc[empty_alfresco_study_code, 'study_name']
    
    study_exception_indexes = df_mlwh_study_exceptions.index[
        df_mlwh_study_exceptions.index.isin(df_return.index)
    ]
    df_return.loc[study_exception_indexes, 'alfresco_study_code'] = df_mlwh_study_exceptions.loc[study_exception_indexes, 'alfresco_study_code']

    # Change any incorrect taxon IDs
    taxon_exception_indexes = df_mlwh_exceptions_this_taxon.index[
        df_mlwh_exceptions_this_taxon.index.to_series().isin(df_return.index)
    ]
    df_return.loc[taxon_exception_indexes, 'taxon_id'] = df_mlwh_exceptions_this_taxon.loc[taxon_exception_indexes, 'taxon_id']

    # Determine sample ID and change any incorrect sample IDs
    df_return['derivative_sample_id'] = df_return.apply(set_sample_id, 1)
    sample_exception_indexes = df_mlwh_sample_exceptions.index[
        df_mlwh_sample_exceptions.index.isin(df_return.index)
    ]    
    df_return.loc[sample_exception_indexes, 'derivative_sample_id'] = 'DS_' + df_mlwh_sample_exceptions.loc[sample_exception_indexes, 'sample_id']
    df_return.drop(['sample', 'name'], axis=1, inplace=True)
    
    # Remove any samples that are not in this taxon
    df_return = df_return.loc[~df_return.index.isin(df_mlwh_exceptions_other_taxa.index)]
    
    # Merge in missing exceptions
    df_return = df_return.append(df_mlwh_missing_exceptions.drop('study_group', 1))
    
    # Remove any samples that don't have an alfresco study
    invalid_codes = ~df_return['alfresco_study_code'].apply(is_valid_alfresco_code)
    if(np.count_nonzero(invalid_codes) > 0):
        print('Determined %d files with invalid alfresco_study_code:' % np.count_nonzero(invalid_codes))
        print(df_return['alfresco_study_code'].loc[invalid_codes].value_counts())
        df_return = df_return.loc[invalid_codes]
    else:
        print("No files with invalid alfresco_study_code")

    # Determine filename in subtrack (for very old samples, there is only a .srf file in subtrack)
    # Note we determined the id_run cutoff (5750) for bam/srf after looking at previous runs. There seems
    # to be a grey area between runs 5500 and 5750 where some files are srf and some are bam
    df_return['subtrack_filename'] = df_return.index
    df_return.loc[df_return['id_run'] < 5750, 'subtrack_filename'] = df_return.loc[df_return['id_run'] < 5750, 'subtrack_filename'].apply(lambda x: x.replace('.bam', '.srf'))
    
    # Read in data from substrack, most importantly to get run accessions
    sql_query = "\
    SELECT \
        files.file_name as irods_filename, \
        files.bytes as subtrack_files_bytes, \
        files.timestamp as subtrack_files_timestamp, \
        files.public_date as subtrack_files_public_date, \
        submission.id as subtrack_submission_id, \
        submission.created as subtrack_submission_created, \
        submission.release_date as subtrack_submission_release_date, \
        submission.timestamp as subtrack_submission_timestamp, \
        submission.ext_db as subtrack_submission_ext_db, \
        submission.ebi_sample_acc, \
        submission.ebi_exp_acc, \
        submission.ebi_study_acc, \
        submission.ebi_sub_acc, \
        submission.ebi_run_acc \
    FROM \
        submission, \
        files \
    WHERE \
        files.sub_id = submission.id AND \
        files.file_name in (%s)\
    " % ("'" + "', '".join(df_return['subtrack_filename']) + "'")

    conn= pymysql.connect(
        host='shap',
        user='ega_dataset',
        password='ega_dataset',
        db='subtrack',
        port=3303
    )

    df_run_accessions = pd.read_sql(sql_query, conn).set_index('irods_filename')
    
    # Merge in subtrack data
    df_return = df_return.join(df_run_accessions, on='subtrack_filename', how='left')

    # Remove unwanted studies
    df_return = df_return.loc[~df_return['alfresco_study_code'].isin(studies_to_exclude)]

    # Create other columns
    df_return.rename(columns={'alfresco_study_code': 'study', 'paired_read': 'paired'}, inplace=True)
    df_return['lane'] = df_return.index.to_series().apply(lambda x: x.split('.')[0])
    df_return['path'] = df_return.apply(lambda x: "%s/%s" % (input_dir, x.name), 1)
    df_return['reads'] = df_return['num_reads'].fillna(-1).astype(int)
    df_return['paired'] = 1
    df_return['irods_path'] = df_return.apply(lambda x: "/seq/%d/%s" % (x['id_run'], x.name), 1)
    df_return['sample'] = df_return['derivative_sample_id'].str.replace('DS_', '')

    # Remove lanes with zero reads
    df_return = df_return.loc[df_return['reads'] != 0]
    
    # Remove control samples (see email from Sonia 03/05/2018 12:20)
    df_return = df_return.loc[df_return['sample'].str.slice(0, 1) != 'C']
    
    # Restrict to certain columns
    if output_columns is not None:
        df_return = (
            df_return[output_columns]
            .sort_values(['study', 'sample', 'id_run', 'position', 'tag_index'])
        )
    
    print(df_return.shape)

    return(df_return) 


In [4]:
df_lanelets_with_invalid_codes = create_build_manifest()

Determined 63 files with invalid alfresco_study_code:
AT-rich amplification                                 53
Test sequencing of high level human contamination.     6
Plasmodium falciparum genome variation 1               4
Name: alfresco_study_code, dtype: int64
(63, 26)


In [5]:
pd.options.display.max_columns = 50
print(df_lanelets_with_invalid_codes.shape)
df_lanelets_with_invalid_codes[0:3]

(63, 26)


,path,study,sample,lane,reads,paired,irods_path,sanger_sample_id,taxon_id,study_lims,study_name,id_run,position,tag_index,qc_complete,manual_qc,description,instrument_name,instrument_model,forward_read_length,requested_insert_size_from,requested_insert_size_to,human_percent_mapped,subtrack_filename,subtrack_files_bytes,ebi_run_acc
irods_filename,,,,,,,,,,,,,,,,,,,,,,,,,,
8002_4#7.bam,/lustre/scratch118/malaria/team112/pipelines/s...,AT-rich amplification,3D7-MspJ-dig,8002_4#7,5719248,1,/seq/8002/8002_4#7.bam,None,36329,1935,AT-rich amplification,8002,4,7.0,2012-06-11 13:52:53,1.0,qc complete,HS28,HiSeq,75,150.0,600.0,72.02,8002_4#7.bam,NaN,None
8002_4#4.bam,/lustre/scratch118/malaria/team112/pipelines/s...,AT-rich amplification,3D7-no-dig,8002_4#4,92952,1,/seq/8002/8002_4#4.bam,None,36329,1935,AT-rich amplification,8002,4,4.0,2012-06-11 13:52:53,1.0,qc complete,HS28,HiSeq,75,150.0,600.0,82.98,8002_4#4.bam,NaN,None
5408_5_nonhuman.bam,/lustre/scratch118/malaria/team112/pipelines/s...,AT-rich amplification,3D7_150,5408_5_nonhuman,25650602,1,/seq/5408/5408_5_nonhuman.bam,None,36329,1935,AT-rich amplification,5408,5,NaN,2010-11-12 22:37:15,1.0,qc complete,IL19,HK,76,0.0,0.0,56.19,5408_5_nonhuman.srf,NaN,NaN


In [6]:
df_lanelets_with_invalid_codes['study'].value_counts(dropna=False)

AT-rich amplification                                 53
Test sequencing of high level human contamination.     6
Plasmodium falciparum genome variation 1               4
Name: study, dtype: int64

In [7]:
df_output = df_lanelets_with_invalid_codes[
    ['study', 'study_lims', 'sample', 'qc_complete', 'description', 'instrument_model', 'instrument_model', 'forward_read_length',
     'requested_insert_size_from', 'ebi_run_acc', 'lane']
].sort_values('lane')

In [8]:
df_output

,study,study_lims,sample,qc_complete,description,instrument_model,instrument_model,forward_read_length,requested_insert_size_from,ebi_run_acc,lane
irods_filename,,,,,,,,,,,
12135_6#66.cram,Test sequencing of high level human contaminat...,2876,PD0338-C,2014-02-28 12:53:57,qc complete,HiSeq,HiSeq,100,200.0,ERR490344,12135_6#66
12135_7#67.cram,Test sequencing of high level human contaminat...,2876,QE0196-C,2014-02-28 12:53:57,qc complete,HiSeq,HiSeq,100,200.0,ERR490345,12135_7#67
12135_7#68.cram,Test sequencing of high level human contaminat...,2876,QE0008-C,2014-02-28 12:53:57,qc complete,HiSeq,HiSeq,100,200.0,ERR490346,12135_7#68
12135_7#69.cram,Test sequencing of high level human contaminat...,2876,QE0145-C,2014-02-28 12:53:57,qc complete,HiSeq,HiSeq,100,200.0,ERR490347,12135_7#69
12135_7#70.cram,Test sequencing of high level human contaminat...,2876,QE0022-C,2014-02-28 12:53:57,qc complete,HiSeq,HiSeq,100,200.0,ERR490348,12135_7#70
12135_7#71.cram,Test sequencing of high level human contaminat...,2876,QE0212-C,2014-02-28 12:53:57,qc complete,HiSeq,HiSeq,100,200.0,ERR490349,12135_7#71
3530_8_nonhuman.bam,AT-rich amplification,1935,PK76-C_200,2009-08-26 00:56:41,qc complete,HK,HK,76,150.0,ERR012749,3530_8_nonhuman
5408_2_nonhuman.bam,AT-rich amplification,1935,PK0076,2010-11-12 22:37:15,qc complete,HK,HK,76,0.0,NaN,5408_2_nonhuman
5408_5_nonhuman.bam,AT-rich amplification,1935,3D7_150,2010-11-12 22:37:15,qc complete,HK,HK,76,0.0,NaN,5408_5_nonhuman


In [9]:
df_output.to_csv('lanelets_with_invalid_codes.txt', sep='\t')

# Conclusion
Only three of the files in Magnus's list of 13 are in an "invalid" study. We will need to look elsewhere for the other 10.